# Automatically generating object masks with SAM

This notebook shows how to segment objects from an image using the Segment Anything Model (SAM). 

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator. 

The notebook is adapted from [segment-anything/notebooks/automatic_mask_generator_example.ipynb](https://github.com/opengeos/segment-anything/blob/pypi/notebooks/automatic_mask_generator_example.ipynb) and [segment geospatial by Prof Quisheng Wu](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/automatic_mask_generator.ipynb).

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [ ]:
# %pip install segment-geospatial leafmap localtileserver

In [1]:
import os
import leafmap
from samgeo import SamGeo, show_image, download_file, overlay_images, tms_to_geotiff

## Create an interactive map

In [3]:
m = leafmap.Map(center=[-1.4358,36.7596], zoom=17, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[-1.4358, 36.7596], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…


Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [9]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [36.7596, -1.4358, 37.0356, -1.2127]

In [10]:
m.user_roi_bounds()

[36.6762, -1.4639, 36.7268, -1.4197]

## Download a sample image

In [11]:
image = "satellite.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=17, source="Satellite", overwrite=True)

Downloaded image 001/342
Downloaded image 002/342
Downloaded image 003/342
Downloaded image 004/342
Downloaded image 005/342
Downloaded image 006/342
Downloaded image 007/342
Downloaded image 008/342
Downloaded image 009/342
Downloaded image 010/342
Downloaded image 011/342
Downloaded image 012/342
Downloaded image 013/342
Downloaded image 014/342
Downloaded image 015/342
Downloaded image 016/342
Downloaded image 017/342
Downloaded image 018/342
Downloaded image 019/342
Downloaded image 020/342
Downloaded image 021/342
Downloaded image 022/342
Downloaded image 023/342
Downloaded image 024/342
Downloaded image 025/342
Downloaded image 026/342
Downloaded image 027/342
Downloaded image 028/342
Downloaded image 029/342
Downloaded image 030/342
Downloaded image 031/342
Downloaded image 032/342
Downloaded image 033/342
Downloaded image 034/342
Downloaded image 035/342
Downloaded image 036/342
Downloaded image 037/342
Downloaded image 038/342
Downloaded image 039/342
Downloaded image 040/342


You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [13]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=1057525.0, center=[-1.4673726777278078, 36.72772407531739], controls=(ZoomControl(options=['positio…

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

In [14]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint='sam_vit_h_4b8939.pth',
    sam_kwargs=None,
)

Checkpoint /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth does not exist.


Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
To: /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.56G/2.56G [00:20<00:00, 126MB/s]


## Automatic mask generation

Segment the image and save the results to a GeoTIFF file. Set `unique=True` to assign a unique ID to each object. 

In [17]:
sam.generate(image, output="masks.tif", foreground=True, unique=True)

OutOfMemoryError: ignored

In [ ]:
sam.show_masks(cmap="binary_r")

Show the object annotations (objects with random color) on the map.

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")

Compare images with a slider.

In [19]:
leafmap.image_comparison(
    "satellite.tif",
    # "annotations.tif",
    label1="Satellite Image",
    # label2="Image Segmentation",
)

TypeError: ignored

Add image to the map.

In [ ]:
m.add_raster("annotations.tif", alpha=0.5, layer_name="Masks")
m

Convert the object annotations to vector format, such as GeoPackage, Shapefile, or GeoJSON.

In [ ]:
sam.tiff_to_vector("masks.tif", "masks.gpkg")

## Automatic mask generation options

There are several tunable parameters in automatic mask generation that control how densely points are sampled and what the thresholds are for removing low quality or duplicate masks. Additionally, generation can be automatically run on crops of the image to get improved performance on smaller objects, and post-processing can remove stray pixels and holes. Here is an example configuration that samples more masks:

In [ ]:
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.86,
    "stability_score_thresh": 0.92,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 100,
}

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint=checkpoint,
    sam_kwargs=sam_kwargs,
)

In [ ]:
sam.generate(image, output="masks2.tif", foreground=True)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", opacity=1, output="annotations2.tif")

Compare images with a slider.

In [ ]:
leafmap.image_comparison(
    image,
    "annotations.tif",
    label1="Image",
    label2="Image Segmentation",
)

Overlay the annotations on the image and use the slider to change the opacity interactively.

In [ ]:
overlay_images(image, "annotations2.tif", backend="TkAgg")

![](https://i.imgur.com/I1IhDgz.gif)